In [1]:
import pandas as pd
import numpy as np

import neologdn
import MeCab

import re

from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

import gensim
from gensim import corpora# 辞書作成用
from gensim import models# LDA用

C:\Users\Tomori kengo\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


### 日誌サンプルデータの読み込み

In [2]:
#encoding='utf-8'で上手くいった
df = pd.read_csv('posi_nega_alldata.csv', encoding='utf-8')
df.head()

,text,posi_nega
0,案件個別の技術相談乗ってます！乗ります,1
1,勉強とお仕事が有機的に繋がっていて面白い。,1
2,次々とテクニックを教えていただき、ためになりました,1
3,お肉がキレイに焼けたことに小さなしあわせを感じた,1
4,これが私の最適化,1


### MeCabにかける前準備としてneologdn.normalize()を使用して文章全体を正規化

In [3]:
def get_diary_normalization(text):
    diary_normalization = neologdn.normalize(text)
    return diary_normalization

### MeCab + neologdで形態素解析し、名詞、形容詞原形と動詞原形を抽出しリストに格納
##### 参考：https://github.com/kujirahand/book-mlearn-gyomu/blob/master/src/ch4/Doc2Vec/create_model.py

In [4]:
neologd_tagger = MeCab.Tagger('-Ochasen -d C:\mecab-ipadic-neologd')

# 引数のテキストを分かち書きして配列にする
# node.surface: 文字のみ取得できる 出力例：同  期間  の  SBI  証券
# node.feature: 品詞、原形などの詳細を取得できる:「品詞,品詞細分類1,品詞細分類2,品詞細分類3,活用形,活用型,原形,読み,発音」の順
# よって、「品詞」を取得したい場合はnode.featureをsplit()でリスト型にした後に抽出したい詳細のインデックス番号[0]を指定すればよい
# また、動詞や形容詞の「原形（の単語）」を取得したい場合はインデックス番号[6]を指定すればよい

def split_words(diary_normalization):
    node = neologd_tagger.parseToNode(diary_normalization) #parseだとエラー「 'str' object has no attribute 'feature'」
    wakati_words = []
    while node is not None:
        hinshi = node.feature.split(",")[0]
        if  hinshi in ["名詞"]:
            wakati_words.append(node.surface)
        elif hinshi in ["形容詞"]:
            wakati_words.append(node.feature.split(",")[6])
        node = node.next
    return wakati_words

In [5]:
wakati_list = []

# 文章の上から順に作成した関数を実行
for diary in df['text']:
    diary_normalization = get_diary_normalization(diary)
    wakati_words = split_words(diary_normalization)
    wakati_list.append(wakati_words)
    
wakati_list

[['案件', '個別', '技術', '相談'],
 ['勉強', 'お仕事', '有機的', '面白い'],
 ['テクニック', 'ため'],
 ['お肉', 'キレイ', 'こと', 'しあわせ'],
 ['これ', '私', '最適化'],
 ['データ', '抽出', 'グラフ', '化', '問題', 'さ', 'そう'],
 ['PC', '復帰', 'みんな', '日報', 'コメント'],
 ['久々', '新卒', '自社', '嬉しい'],
 ['おでん', '美味しい', '季節'],
 ['昼飯', '割高', '値段', '満足感'],
 ['10歳', '以上', '若い', '判定'],
 ['2週', '連続', '出展'],
 ['参加', '皆様', '皆様'],
 ['無事', 'こと'],
 ['これ', '協力', '皆様', 'おかげ'],
 ['本当にありがとうございました'],
 ['本', 'モデル', '会社', 'サービス', 'ビジネスモデル', '歴史', '軽い', '紹介', 'おもしろい'],
 ['ベイズ', '統計', 'モデリング', 'こと'],
 ['休日', 'オフィス', '出社', '静か', 'さ', '良い'],
 ['すごい', 'いい', '話', '気'],
 ['正直', '精度', '自体', 'よい'],
 ['常駐', '先', '上長', 'メンタル', '辛い', 'そう'],
 ['期日'],
 ['打ち合わせ', '中', '体調', '悪い'],
 ['結局', '僕', '凡', 'ミス', '原因'],
 ['家'],
 ['仲', '悪い', 'の', 'よう', 'やり取り', 'みんな', '大丈夫', '元気'],
 ['変化', '気', 'よう', '期待', '損'],
 ['ドローン', '人間', '事', '何', 'よう', '宣伝', 'ヤツ', 'の', 'マジで', '迷惑', 'ん'],
 ['案件', '業務', '内容', '統一', '評価基準', '用意', 'の', '難しい'],
 ['良い', '子', '大人'],
 ['先月', '案件', '継続', '確認', '月末', 'イレギュラー', '多い'

## LDA用の辞書作成
使用単語をリスト化したものを辞書と呼び、gensimの型の辞書を作成

In [6]:
# 参考URL：https://pira-nino.hatenablog.com/entry/2018/07/29/B%27z%E3%81%AE%E6%AD%8C%E8%A9%9E%E3%82%92Python%E3%81%A8%E6%A9%9F%E6%A2%B0%E5%AD%A6%E7%BF%92%E3%81%A7%E5%88%86%E6%9E%90%E3%81%97%E3%81%A6%E3%81%BF%E3%81%9F_%E3%80%9CLDA%E7%B7%A8%E3%80%9C
# from gensim import corpora

dictionary =corpora.Dictionary(wakati_list)

# dictionary.filter_extremes(no_below=3, no_above=0.3) で特徴語として相応しくない単語を削除
# no_below で指定された数値より下回る頻度で出現した単語は特徴語としてみなされず、また逆に no_above で書かれた割合より多く出現している単語は除かれます。
# dictionary.filter_extremes(no_above=0.8)

## Corpus作成
作成した辞書を元に、gensimのLDAの入力の型に文書データを変換
doc2bowという関数を使用

In [7]:
corpus=[dictionary.doc2bow(tokens) for tokens in wakati_list]

## LDAの学習
作成辞書とコーパスを使ってLDAを回す

In [8]:
#トピック数の設定
topic_num=2
#モデルの学習
model = gensim.models.LdaModel(corpus,
                               num_topics=topic_num,
                               id2word=dictionary,
                               random_state=0
                              )

model.save('lda_test.model')

In [9]:
# model.print_topic(i)でi番のトピックを取得
# for文でトピックを出力
for i in range(topic_num):
    print('TOPIC:', i, '→', model.print_topic(i))
    print('------------------------------------------')

TOPIC: 0 → 0.015*"嬉しい" + 0.015*"の" + 0.015*"気" + 0.014*"こと" + 0.013*"案件" + 0.013*"これ" + 0.011*"よう" + 0.011*"皆様" + 0.010*"いい" + 0.010*"確認"
------------------------------------------
TOPIC: 1 → 0.017*"よう" + 0.013*"こと" + 0.012*"悪い" + 0.012*"の" + 0.012*"嬉しい" + 0.012*"みんな" + 0.009*"これ" + 0.009*"そう" + 0.009*"時間" + 0.009*"以上"
------------------------------------------


In [10]:
# 上記を見やすく出力
# model.show_topic(i)でi番のトピックを取得
# http://lab.astamuse.co.jp/entry/try-lda
for i in range(topic_num):
    print("\n")
    print("="*80)
    print("TOPIC {0}\n".format(i))
    topic = model.show_topic(i)
    for t in topic:
        print("{0:20s}{1}".format(t[0], t[1]))



TOPIC 0

嬉しい                 0.015400723554193974
の                   0.015278492122888565
気                   0.014543972909450531
こと                  0.01413735467940569
案件                  0.013479954563081264
これ                  0.013404465280473232
よう                  0.011225086636841297
皆様                  0.010529246181249619
いい                  0.010475586168467999
確認                  0.009767688810825348


TOPIC 1

よう                  0.016998596489429474
こと                  0.013443437404930592
悪い                  0.012392902746796608
の                   0.012050390243530273
嬉しい                 0.01190117560327053
みんな                 0.011848006397485733
これ                  0.009221171960234642
そう                  0.009022468701004982
時間                  0.008878090418875217
以上                  0.008712693117558956
